In [3]:
using StaticArrays
using LinearAlgebra
using Combinatorics
sa = StaticArrays
la = LinearAlgebra
cm = Combinatorics 

Combinatorics

In [7]:
SPoint = sa.SVector{2, Float64}
SamePoints = Tuple{SPoint, SPoint, SPoint, SPoint}
function first_in_klein(point::SPoint)::SPoint
    x::Float64 = point[1]
    y::Float64 = point[2]
    if (0 <= x <= 1) & (0 <= y <= 1)
        return point

    elseif -1 <= x < 0 < y <= 1
        return @sa.SVector [x + 2.0, 2.0 - y]

    elseif (-1 <= x < 0) & (-1 <= y <= 0)
        return @sa.SVector [x + 2.0, -y]

    elseif (-1 <= y < 0 <= x <=1)
        return @sa.SVector [x, 2.0 + y]
    end
    throw(DomainError(point, "Make sure that the first and second coordinates are in the intreval [-1, 1]"))
end

function samepoints_in_kelin(point:: SPoint)::SamePoints
    firstpoint:: SPoint = first_in_klein(point)
    x:: Float64 = firstpoint[1]
    y:: Float64 = firstpoint[2]
    second:: SPoint = @sa.SVector [x - 2.0, 2.0 - y]
    theard:: SPoint = @sa.SVector [x - 2.0, -y]
    fourth:: SPoint =  @sa.SVector [x, y - 2.0]
    return (firstpoint, second, theard, fourth)
end


samepoints_in_kelin (generic function with 1 method)

In [5]:
point = @sa.SVector [0.5, -1.0]
println(samepoints_in_kelin(point))

([0.5, 1.0], [-1.5, 1.0], [-1.5, -1.0], [0.5, -1.0])


In [6]:
Quadrants = Tuple{Int64, Int64}

function quadrants_square_distance(quadrants:: Quadrants, samepoints1:: SamePoints, samepoints2:: SamePoints)::Float64
    index1:: Int64 = quadrants[1]
    index2:: Int64 = quadrants[2]
    diff:: SPoint = samepoints1[index1]- samepoints2[index2]
    return la.norm(diff)
end

function candidates_quadrants(point1:: SPoint, point2:: SPoint)::Tuple{Quadrants, Quadrants}
    add_second:: Float64 = (point1 + point2)[2]
    diff:: SPoint = point1 - point2
    #print(diff[2], 'in candidate_quadrants')
    #candidate1:: Quadrants = (1, 3) if (add_second < 1.0)  else (1, 2) if (add_second < 3.0) else (4, 2)
    candidate1:: Quadrants = add_second < 1.0 ? (1, 3) : add_second < 3.0 ? (1, 2) : (4, 2)
    candidate2:: Quadrants =  diff[2] < -1.0 ? (1, 4) : diff[2] < 1.0 ? (1, 1) : (2, 3)
    return (diff[1] >= 0 ? reverse(candidate1) : candidate1, candidate2)
end


candidates_quadrants (generic function with 1 method)

In [8]:

function distance_in_klein_bottle(point1:: SPoint, point2:: SPoint)::Float64
    samepoints1:: SamePoints = samepoints_in_kelin(point1)
    samepoints2:: SamePoints = samepoints_in_kelin(point2)
    point1_in_first:: SPoint = samepoints1[1]
    point2_in_first:: SPoint = samepoints2[1]
    candidates:: Tuple{Quadrants, Quadrants} = candidates_quadrants(point1_in_first, point2_in_first)
    #print(candidates, 'in dist klein')
    dist1:: Float64 = quadrants_square_distance(candidates[1], samepoints1, samepoints2)
    dist2:: Float64 = quadrants_square_distance(candidates[2], samepoints1, samepoints2)
    #print(dist1, dist2)
    return minimum((dist1, dist2))
end

function distance_map(pointcloud)
    return map(points -> distance_in_klein_bottle(points[1], points[2]), cm.combinations(pointcloud, 2))
end


distance_map (generic function with 1 method)

In [9]:
function uniformpoints_in_square(;range_min:: Float64, range_max:: Float64, sampling:: Int, dim:: Int64 =1,  step:: Float64 = 0.00001)
    rng = range_min:step:range_max
    return rand(rng, (sampling, dim))
end

function staticalize(ndarray:: Array{Float64, 2})
    rownum ::Int = size(ndarray, 1)
    return Tuple(SPoint(ndarray[i,:]) for i in 1:rownum)
end

staticalize (generic function with 1 method)

In [16]:
pointcloud = uniformpoints_in_square(range_min=-1.0, range_max=1.0, sampling=1000, dim = 2)
staticpointcloud = staticalize(pointcloud)
staticpointcloud[1]

2-element SArray{Tuple{2},Float64,1,2} with indices SOneTo(2):
 0.04389
 0.40414

In [15]:
point1 = @sa.SVector [0.0, -1.0]
point2 = @sa.SVector [0.0, 1.0]
println(distance_in_klein_bottle(point, point))

0.0


In [17]:
distance_map(staticpointcloud)

499500-element Array{Float64,1}:
 0.604659890682357
 1.0883113066122214
 1.0805292682755059
 0.8965185201656459
 1.0424766937442775
 0.7629218393780585
 0.4250459261068149
 0.8920391956074577
 0.6901893169268849
 0.6281660527280983
 0.9938169627250282
 0.6863369211254775
 1.0688360235321412
 ⋮
 0.4592378368558061
 1.0050031965123294
 0.6636185374897237
 0.3526762919448939
 0.61558470578792
 0.17655004106484928
 0.9997760359200453
 0.8662099196499655
 0.5036757874863551
 0.8352657539370328
 0.5292259597752174
 0.5597461047653658